In [1]:
import pandas as pd
import numpy as np
#import matplotlib
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/Users/eyobmanhardt/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
df = pd.read_csv('temp_preprocess_data.csv')

In [4]:
print('shape:', df.shape)
df.head()

(356251, 131)

In [12]:
train_df = df.loc[df['TARGET'].notnull(), :]
test_df = df.loc[df['TARGET'].isnull(), :]
test_df.drop(columns= ['TARGET'], axis=1, inplace=True)

train_df_enc = pd.get_dummies(train_df)
test_df_enc = pd.get_dummies(test_df)
print('train shape:', train_df_enc.shape) 
print('test shape:', test_df_enc.shape) 

/Users/eyobmanhardt/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


train shape: (307507, 251)
test shape: (48744, 248)


In [15]:
train_df_enc_cols = set(train_df_enc.columns)
test_df_enc_cols = set(test_df_enc.columns)

print(train_df_enc_cols.difference(test_df_enc_cols))

{'NAME_INCOME_TYPE_Maternity leave', 'NAME_FAMILY_STATUS_Unknown', 'TARGET'}


In [16]:
y = train_df_enc['TARGET']
## align dataframes 
X, X_test = train_df_enc.align(test_df_enc, join='inner', axis=1)

In [17]:
X.shape, X_test.shape, y.shape

((307507, 248), (48744, 248))

(307507,)

In [26]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='median')

In [21]:
Test_ids = X_test['SK_ID_CURR']
X.drop(columns= ['SK_ID_CURR'], inplace=True)
X_test.drop(columns= ['SK_ID_CURR'], inplace=True)

In [25]:
X.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,0,1,0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0,0,0,0,0,0,1,0,1,0
1,1,1,1,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0,1,0,0,0,0,0,0,1,0
2,0,0,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0,0,0,0,0,0,0,0,0,0


In [27]:
## impute missing values by its median 
imp.fit(X)
X = imp.transform(X)
X_test = imp.fit_transform(X_test)

/Users/eyobmanhardt/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: Input contains infinity or a value too large for dtype('float64').

In [28]:
np.isinf(X).values.sum()

4142

In [29]:
X.columns.to_series()[np.isinf(X).any()]

AMT_CREDIT_DEBT_RATIO    AMT_CREDIT_DEBT_RATIO
dtype: object

In [33]:
X['AMT_CREDIT_DEBT_RATIO']

0         0.068272
1         0.000000
2         0.000000
3              NaN
4         0.000000
            ...   
307502         NaN
307503         NaN
307504    0.299308
307505    0.000000
307506    0.296100
Name: AMT_CREDIT_DEBT_RATIO, Length: 307507, dtype: float64

In [36]:
X['AMT_CREDIT_DEBT_RATIO'] = X['AMT_CREDIT_DEBT_RATIO'].apply(lambda x: x if ~np.isinf(x) else 0)
np.isinf(X).values.sum()

0

In [37]:
np.isinf(X_test).values.sum()

236

In [38]:
X_test.columns.to_series()[np.isinf(X_test).any()]

AMT_CREDIT_DEBT_RATIO    AMT_CREDIT_DEBT_RATIO
dtype: object

In [39]:
X_test['AMT_CREDIT_DEBT_RATIO'] = X_test['AMT_CREDIT_DEBT_RATIO'].apply(lambda x: x if ~np.isinf(x) else 0)

In [40]:
np.isinf(X_test).values.sum()

0

In [43]:
X[y==0].shape

(282682, 247)

In [44]:
X[y==1].shape

(24825, 247)

In [41]:
## impute missing values by its median 
imp.fit(X)
X = imp.transform(X)
X_test = imp.fit_transform(X_test)

# down-sampling

In [46]:
class0 = X[y==0]
class1 = X[y==1]
print(class0.shape, class1.shape)

(282682, 247) (24825, 247)


In [62]:
diff = train_df_enc_cols.difference(test_df_enc_cols)
cols = [c for c in train_df_enc.columns if c not in diff]

In [63]:
cols[0]

'SK_ID_CURR'

In [65]:
df_prime = pd.DataFrame(X, columns = cols[1:])
df_prime['TARGET'] = y
df_prime.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET
0,0.0,1.0,0.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
1,1.0,1.0,1.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
df_class0 = df_prime.loc[df_prime['TARGET']==0, :]
df_class1 = df_prime.loc[df_prime['TARGET']==1, :]

In [70]:
df_class0.shape, df_class1.shape

((282682, 248), (24825, 248))

In [72]:
df_class0_down_sample = df_class0.sample(df_class1.shape[0])

In [74]:
df_class0_down_sample.shape, df_class1.shape

((24825, 248), (24825, 248))

In [75]:
df_ds = pd.concat([df_class0_down_sample, df_class1], axis = 0)

In [76]:
df_ds.shape

(49650, 248)

In [78]:
df_ds.TARGET.value_counts()

1.0    24825
0.0    24825
Name: TARGET, dtype: int64

In [79]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier

In [80]:
X = df_ds.drop(columns = ['TARGET'], axis=1)
y = df_ds['TARGET']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state= 42, stratify=y)

# up-sampling 
# SMOTE

In [56]:
3 not in [1,2,3]

False